# Tested on ml.p4de.24xlarge

In [3]:
!nvidia-smi

Thu Dec 21 04:18:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:10:1C.0 Off |                    0 |
| N/A   19C    P0    55W / 400W |      0MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:10:1D.0 Off |                    0 |
| N/A   

In [ ]:
# Matches the CUDA version reported by nvidia-smi (11.8, in this case)'
# Configure here: https://pytorch.org/
%pip install -U torch --index-url https://download.pytorch.org/whl/cu118

%pip install -U transformers==4.36.2
%pip install -U peft==0.7.1
%pip install -U datasets==2.15.0
%pip install -U bitsandbytes==0.41.2
%pip install -U scipy==1.10.1
%pip install -U ipywidgets=-8.1.1
%pip install -U matplotlib==3.7.4

In [84]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

Detected kernel version 4.14.327, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [85]:
from datasets import load_dataset

dataset_name = "databricks/databricks-dolly-15k"

train_dataset = load_dataset(dataset_name, split="train[0:800]")
eval_dataset = load_dataset(dataset_name, split="train[800:1000]")

In [178]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mixtral-8x7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [179]:
# Tokenization 
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

# def tokenize(prompt):
#     result = tokenizer(prompt)
#     result["labels"] = result["input_ids"].copy()
#     return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = f"""

Given a question and some additional context, provide an answer.

### Question:
{data_point['instruction']}

### Context:
{f"Here is some context: {data_point['context']}" if len(data_point["context"]) > 0 else ""}

### Response:
{data_point['response']}

"""

    tokenized_prompt = tokenizer(full_prompt)
    return tokenized_prompt

tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

untokenized_text = tokenizer.decode(tokenized_train_dataset[1]['input_ids']) 
print(untokenized_text)

<s> 

Given a question and some additional context, provide an answer.

### Question:
Which is a species of fish? Tope or Rope

### Context:


### Response:
Tope

</s>


In [ ]:
max_length = 400 

# def tokenize(prompt):
#     result = tokenizer(
#         prompt,
#         truncation=True,
#         max_length=max_length,
#         padding="max_length",
#     )
#     result["labels"] = result["input_ids"].copy()
#     return result

tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

untokenized_text = tokenizer.decode(tokenized_train_dataset[4]['input_ids']) 
print(untokenized_text)

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "w1",
        "w2",
        "w3",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Apply the accelerator. You can comment this out to remove the accelerator.
model = accelerator.prepare_model(model)

In [ ]:
!rm -rf ./dolly_mixtral_finetune

In [ ]:
import transformers
from datetime import datetime

if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

output_dir = "./dolly_mixtral_finetune"

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=1,
        gradient_checkpointing=True,
        gradient_accumulation_steps=4,
        max_steps=10,
        learning_rate=2.5e-5, 
        logging_steps=5,
        fp16=True, 
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=10,               # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=10,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mixtral-8x7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id, 
    quantization_config=bnb_config,
    device_map="auto"
)

# Load model and perform inference

In [ ]:
from peft import PeftModel

peft_model = PeftModel.from_pretrained(base_model, f"{output_dir}/checkpoint-10")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
#    add_bos_token=True
)
tokenizer.pad_token = tokenizer.eos_token

# eval_tokenizer = AutoTokenizer.from_pretrained(
#     base_model_id,
# #    trust_remote_code=True,
#     add_bos_token=True,
# )

In [ ]:
prompt = """

Given a question and some additional context, provide an answer.

### Question:
Who wrote "Generative AI on AWS" by O'Reilly Media?

### Context: 
Chris Fregly and Antje Barth wrote "Data Science on AWS" by O'Reilly Media.
Chris Fregly, Antje Barth, and Shelbee Eigenbrode wrote "Generative AI on AWS" by O'Reilly Media.

### Response:

"""

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

peft_model.eval()

with torch.no_grad():
    print(tokenizer.decode(peft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))